In [ ]:
%pip -q install transformers
%pip -q install sentencepiece
%pip -q install accelerate

In [ ]:
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer

# Hugging Face model_path
model_path = 'psmathur/orca_mini_3b'
tokenizer = LlamaTokenizer.from_pretrained(model_path)
model = LlamaForCausalLM.from_pretrained(
    model_path, torch_dtype=torch.float16, device_map='auto',
)


#generate text function
def generate_text(system, instruction, input=None):

    if input:
        prompt = f"### System:\n{system}\n\n### User:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n"
    else:
        prompt = f"### System:\n{system}\n\n### User:\n{instruction}\n\n### Response:\n"

    tokens = tokenizer.encode(prompt)
    tokens = torch.LongTensor(tokens).unsqueeze(0)
    tokens = tokens.to('cuda')

    instance = {'input_ids': tokens,'top_p': 1.0, 'temperature':0.7, 'generate_len': 1024, 'top_k': 50}

    length = len(tokens[0])
    with torch.no_grad():
        rest = model.generate(
            input_ids=tokens,
            max_length=length+instance['generate_len'],
            use_cache=True,
            do_sample=True,
            top_p=instance['top_p'],
            temperature=instance['temperature'],
            top_k=instance['top_k']
        )
    output = rest[0][length:]
    string = tokenizer.decode(output, skip_special_tokens=True)
    return f'[!] Response: {string}'

system = 'You are an AI assistant that follows instruction extremely well. Help as much as you can.'

instruction = '''
  I really hated the show! -- negative\n
  I somewhat liked it -- Positive\n
  Meh, i am not really sure about that show -- neutral\n
  It was ok. I guess.. --
'''
print("Few-shot: \n"+str(generate_text(system, instruction)))


instruction = '''
Which is a faster way to get home?\n
Option 1-Take an 10 minutes bus, then an 40 minute bus, and finally a 10 minute train.\n
Option 2-Take a 90 minutes train, then a 45 minute bike ride, and finally a 10 minute bus.\n
Option 1 will take 10+40+10 = 60 minutes.\n
Option 2 will take 90+45+10 = 145 minutes.\n
Since Option 1 takes 60 minutes and Option 2 takes 145 minutes, Option 1 is less than option 2 which makes it faster.\n

Which is a faster way to get to work?\n
Option 1 - Take a 1000 minute bus, then a half hour train, and finally a 10 minute bike ride.\n
Option 2 - Take an 800 minute bus, then an hour train, and finally a 30 minute bike ride.\n
Take a deep breath and think step by step. Which option is faster?\n
'''
print("Few-Shot COT: \n"+str(generate_text(system, instruction)))